In [1]:
from langchain_community.llms import Ollama

from langchain_core.prompts import ChatPromptTemplate


from langchain_core.output_parsers import StrOutputParser


# llm = Ollama (model = "gemma2:27b") #gemma
llm = Ollama(model = "llama3.1:405b") #llama
output_parser = StrOutputParser()

In [2]:
llm.invoke("hello world?")

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Nurse ."),
    ("user", "{input}")
])

In [4]:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "사용자의 대변 색상 코드: #0000FF, 브리스톨 지수: scale 4, 혈액 존재: 있음이야, 이에 대한 건강 상태와 관련된 의심되는 상황을 해결하기 위해, 나에게 추가적인 질문을 한가지만 해줘"})

'변의 양은 어느 정도인가요? (예: 소량, 보통, 많음) \n\n\n변의 색상과 브리스톨 지수, 그리고 혈액 존재 여부는 중요한 정보이지만 변의 양 또한 장 건강 상태를 평가하는 데 도움이 될 수 있습니다.'

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9791448/")

docs_1 = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model = "gemma2:27b"
)

In [8]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs_1)
vector = FAISS.from_documents(documents, embeddings)

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)